<a href="https://colab.research.google.com/github/ctruciosm/BasicStats/blob/main/AF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Análise Fatorial
By Carlos Trucíos

In [ ]:
install.packages("psych")
install.packages("mnt")
install.packages("POET")

In [ ]:
library(psych)
library(mnt)
library(dplyr)
library(POET)

# 0. Dados

Utilizaremos o [_data set_](https://raw.githubusercontent.com/ctruciosm/ctruciosm.github.io/master/datasets/monthly_data.csv) `monthly_data.csv` que contém os retornos mensais de 16 ações negociadas na bolsa e valores de São Paulo entre Janeiro 2000 até Fevereiro 2022.

In [3]:
# Importar dados
dados = read.csv("https://raw.githubusercontent.com/ctruciosm/ctruciosm.github.io/master/datasets/monthly_data.csv")
head(dados)
dados = scale(as.matrix(dados), center = TRUE, scale = FALSE)

,ALPA4,BBAS3,BBDC3,BBDC4,CMIG4,CPLE6,ELET3,ELET6,EMBR3,GGBR4,ITSA4,LIGT3,PETR3,PETR4,SBSP3,VIVT3
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,-6.35512370,-5.952382,-2.078679,3.200547,-23.950618,-17.714287,-17.047656,-13.722833,-7.888889,4.166667,-8.500541,20.628142,-14.991126,-11.304351,-5.7048544,33.243479
2,2.42718215,-5.696199,-10.708152,-12.924411,-5.487014,12.500005,-4.392524,-7.173605,1.745443,-1.600005,-3.529415,1.641325,26.466875,12.745102,-12.2499986,18.749998
3,9.95260583,-14.248766,9.934533,13.243936,4.706286,-6.507126,1.010101,3.554871,8.795633,-7.520324,3.844540,-9.832372,4.304947,4.449614,0.5697932,-15.444107
4,-5.11890398,-4.610491,-6.916793,-8.119986,-7.685016,-9.084745,-13.870968,-11.044779,6.145254,-4.175821,-8.823535,-5.299998,-9.090907,-8.877120,-3.3982805,-11.151319
5,14.81339362,-4.999999,-4.789383,-6.572207,2.181821,9.619687,8.913859,3.691275,-13.789478,-4.862384,-3.870964,-12.642799,1.249999,-2.580794,-21.9882677,2.184377
6,0.01666724,13.157897,20.656147,27.561080,14.313816,15.646263,27.235216,29.126219,34.736059,8.230900,17.650236,12.631870,30.814813,30.071599,10.5225306,-28.623187


In [4]:
test.BHEP(dados, MC.rep = 1000)

  |======================================================================| 100%



------------------------------------------------------------------------- 

         Test for multivariate normality with the BHEP  teststatistic.

tuning parameter = 1  
BHEP  =  1.10135  
critical value =   1.000111  (via monte carlo) 


------------------------------------------------------------------------- 

## 1. É pertinente aplicar AF?

- Teste de esfericidade de Barlett: $$H_0: \textbf{R} = \textbf{I} \quad vs. \quad H_1: \textbf{R} \neq = \textbf{I}$$ 

Estaística de teste: $-[n - 1 -(2p+5)/6] \log |\textbf{R}| \sim \chi^2_{(p^2-p)/2}$

- Medida de adequação amostral de Kaisser, Meyer e Olkin (**KMO**): 


$$KMO = MSA = \dfrac{\displaystyle \sum \sum_{h \neq j} r_{hj}^2}{\displaystyle \sum \sum_{h \neq j} r_{hj}^2 + \displaystyle \sum \sum_{h \neq j} a_{hj}^2},$$
em que $r_{hj}$ é o coeficiente de correlação entre $X_h$ e $X_j$ e $a_{hj}$ é o coeficiente de correlação parcial entre as variáveis. 

Se o mdelo fatorial pode ser aplicado, $a_{hj}$ será zero ou muito pequeno (pois "retirar"o efeito da outra variáve equivale a retirar o efeito dos fatores comuns), fazendo com que KMO seja próximo de 1 (valores inferiores a 0.5 não são desejáveis).

Baseado no KMO, pode també ser calculado a medida de adequação amostral para cada uma das variáveis.

$$MSA_j = \dfrac{\displaystyle \sum_{h \neq j} r_{hj}^2}{\displaystyle \sum_{h \neq j} r_{hj}^2 + \displaystyle \sum_{h \neq j} a_{hj}^2},$$

Valores próximos a 1 indicam que $X_j$ é adequada para AF.


In [5]:
cortest.bartlett(cor(dados), n = nrow(dados), diag = TRUE) # Requer normalidade

$chisq
[1] 3614.428

$p.value
[1] 0

$df
[1] 120

In [6]:
KMO(dados)

Kaiser-Meyer-Olkin factor adequacy
Call: KMO(r = dados)
Overall MSA =  0.88
MSA for each item = 
ALPA4 BBAS3 BBDC3 BBDC4 CMIG4 CPLE6 ELET3 ELET6 EMBR3 GGBR4 ITSA4 LIGT3 PETR3 
 0.95  0.96  0.87  0.84  0.92  0.93  0.80  0.79  0.87  0.95  0.93  0.95  0.78 
PETR4 SBSP3 VIVT3 
 0.79  0.96  0.92 

## 2. Aplicar AF

- Covariância ou Correlação? (seguir os mesmos critérios do ACP)
- Quantos fatores?
- Qual método de estimação utilizar?
- Qual método para extrair os fatores?

In [7]:
apply(dados, 2, sd)

ALPA4     BBAS3     BBDC3     BBDC4     CMIG4     CPLE6     ELET3     ELET6 
10.971053 12.029736  9.634486  9.780371 10.613703 10.000633 14.603155 13.046453 
    EMBR3     GGBR4     ITSA4     LIGT3     PETR3     PETR4     SBSP3     VIVT3 
11.641115 13.110303  8.631587 13.487736 12.062489 11.996239 10.517618  7.673349

In [8]:
POETKhat(t(dados))

5.036303,5.215364,5.394690,5.574049,5.753488,5.932980,6.112495,6.292081,6.471704,6.651370
5.040173,5.223104,5.406301,5.589530,5.772840,5.956201,6.139586,6.323043,6.506536,6.690072


Segundo o visto na aula, a matriz de cargas fatoriais e os fatores são obtidos da seguinte forma:

$$\hat{\textbf{F}} = \textbf{x} \textbf{P}_1\textbf{D}_1^{-1/2} \quad e \quad  \hat{\Lambda} = \textbf{P}_1 \textbf{D}_1^{1/2}$$


In [9]:
# Manual
dados = scale(dados, scale = TRUE)
eigen_aux <- eigen(cor(dados))
P <- eigen_aux$vectors       
D <- diag(eigen_aux$values) 
F_hat <- dados %*% P[,1] * 1/sqrt(D[1, 1])
Lambda_hat <- P[, 1] * sqrt(D[1, 1])

In [10]:
modelo_af_pca = principal(dados, nfactors = 1, rotate = "none", scores = TRUE)

In [11]:
# As cargas fatoriais feitas na mao e com a implementação batem?
cbind(Lambda_hat, modelo_af_pca$loadings)

,Lambda_hat,PC1
ALPA4,-0.4723331,0.4723331
BBAS3,-0.8230853,0.8230853
BBDC3,-0.8025489,0.8025489
BBDC4,-0.8353218,0.8353218
CMIG4,-0.7525191,0.7525191
CPLE6,-0.7153460,0.7153460
ELET3,-0.7002974,0.7002974
ELET6,-0.6931126,0.6931126
EMBR3,-0.3303562,0.3303562
GGBR4,-0.6789408,0.6789408


In [12]:
# Os fatores (scores) feitos na mão e com a implementação batem?
head(cbind(F_hat, modelo_af_pca$scores))

,,PC1
1,0.89050095,-0.89050095
2,0.19590467,-0.19590467
3,0.07818002,-0.07818002
4,1.17197970,-1.17197970
5,0.48120457,-0.48120457
6,-1.94134826,1.94134826


In [13]:
# Rotação
## Como temos k = 1, não faz sentido fazermos rotação, mas para ilustrar, faremos k = 2
modelo_af_pca_sem_rotar = principal(dados, nfactors = 2, scores = TRUE, rotate = "none")
modelo_af_pca_varimax = principal(dados, nfactors = 2, scores = TRUE, rotate = 'varimax')

In [14]:
modelo_af_pca_sem_rotar$loadings


Loadings:
      PC1    PC2   
ALPA4  0.472       
BBAS3  0.823 -0.179
BBDC3  0.803 -0.308
BBDC4  0.835 -0.307
CMIG4  0.753  0.209
CPLE6  0.715  0.290
ELET3  0.700  0.579
ELET6  0.693  0.584
EMBR3  0.330 -0.209
GGBR4  0.679       
ITSA4  0.848 -0.229
LIGT3  0.601  0.233
PETR3  0.756 -0.257
PETR4  0.801 -0.250
SBSP3  0.616  0.136
VIVT3  0.425       

                 PC1   PC2
SS loadings    7.715 1.335
Proportion Var 0.482 0.083
Cumulative Var 0.482 0.566

In [15]:
modelo_af_pca_varimax$loading


Loadings:
      RC1   RC2  
ALPA4 0.402 0.254
BBAS3 0.750 0.384
BBDC3 0.815 0.272
BBDC4 0.841 0.293
CMIG4 0.449 0.639
CPLE6 0.369 0.678
ELET3 0.174 0.892
ELET6 0.165 0.892
EMBR3 0.388      
GGBR4 0.582 0.361
ITSA4 0.801 0.361
LIGT3 0.317 0.562
PETR3 0.747 0.281
PETR4 0.777 0.315
SBSP3 0.390 0.496
VIVT3 0.330 0.267

                 RC1   RC2
SS loadings    5.145 3.904
Proportion Var 0.322 0.244
Cumulative Var 0.322 0.566

In [75]:
# Outros métodos de estimar as cargas/fatores
modelo_af_fp = fa(dados, nfactors = 1, fm = "pa")
modelo_af_mv = fa(dados, nfactors = 1, fm = "ml")

In [77]:
cbind(modelo_af_pca$loadings, modelo_af_fp$loadings, modelo_af_mv$loading)

,PC1,PA1,ML1
ALPA4,0.4723331,0.4356876,0.4161115
BBAS3,0.8230853,0.8133646,0.8168817
BBDC3,0.8025489,0.7903095,0.8712896
BBDC4,0.8353218,0.8295515,0.9040527
CMIG4,0.7525191,0.7289212,0.6887567
CPLE6,0.7153460,0.6870637,0.6428744
ELET3,0.7002974,0.6699717,0.6019735
ELET6,0.6931126,0.6621171,0.5942171
EMBR3,0.3303562,0.3004060,0.3019552
GGBR4,0.6789408,0.6476274,0.6138870


In [18]:
head(cbind(modelo_af_pca$scores, modelo_af_fp$scores, modelo_af_mv$scores))

,PC1,PA1,ML1
1,-0.89050095,-0.92646362,-0.8069548
2,-0.19590467,-0.39642738,-0.5908011
3,-0.07818002,-0.02051454,0.2440422
4,-1.17197970,-1.13289355,-1.1216091
5,-0.48120457,-0.52742879,-0.5960306
6,1.94134826,2.01540279,2.1362188


# 3. Que tão boa foi nossa AF?

Para responder a esta pergunta focaremos em:

- Cumunalidades
- Diferença entra a correlação observada $\textbf{R}$ e a reproduzida $\hat{\Lambda} \hat{\Lambda}' + \hat{\boldsymbol{\Psi}}$

In [19]:
modelo_af_pca

Principal Components Analysis
Call: principal(r = dados, nfactors = 1, rotate = "none", scores = TRUE)
Standardized loadings (pattern matrix) based upon correlation matrix
       PC1   h2   u2 com
ALPA4 0.47 0.22 0.78   1
BBAS3 0.82 0.68 0.32   1
BBDC3 0.80 0.64 0.36   1
BBDC4 0.84 0.70 0.30   1
CMIG4 0.75 0.57 0.43   1
CPLE6 0.72 0.51 0.49   1
ELET3 0.70 0.49 0.51   1
ELET6 0.69 0.48 0.52   1
EMBR3 0.33 0.11 0.89   1
GGBR4 0.68 0.46 0.54   1
ITSA4 0.85 0.72 0.28   1
LIGT3 0.60 0.36 0.64   1
PETR3 0.76 0.57 0.43   1
PETR4 0.80 0.64 0.36   1
SBSP3 0.62 0.38 0.62   1
VIVT3 0.42 0.18 0.82   1

                PC1
SS loadings    7.71
Proportion Var 0.48

Mean item complexity =  1
Test of the hypothesis that 1 component is sufficient.

The root mean square of the residuals (RMSR) is  0.09 
 with the empirical chi square  551.75  with prob <  3.7e-62 

Fit based upon off diagonal values = 0.96

In [24]:
modelo_af_pca$residual

,ALPA4,BBAS3,BBDC3,BBDC4,CMIG4,CPLE6,ELET3,ELET6,EMBR3,GGBR4,ITSA4,LIGT3,PETR3,PETR4,SBSP3,VIVT3
ALPA4,0.77690144,-0.066627433,-0.058534352,-0.0677730501,-0.059359493,-0.10136733,-0.0422346915,-0.037661461,0.0466589950,-0.023402042,-0.01214240,-0.0464593649,-0.025158099,-0.03484901,0.03955367,0.060282583
BBAS3,-0.06662743,0.322530642,0.018398239,0.0209745280,-0.042142949,-0.05637133,-0.1010602764,-0.097374623,-0.0731883181,0.007064551,0.03304410,-0.0833933717,0.011974459,0.03220912,-0.06802733,-0.017446314
BBDC3,-0.05853435,0.018398239,0.355915338,0.2677344931,-0.071559455,-0.06112774,-0.1381646041,-0.145531732,-0.0074347675,-0.102137130,0.12921416,-0.0948971635,-0.066867936,-0.05712308,-0.08734877,-0.075841845
BBDC4,-0.06777305,0.020974528,0.267734493,0.3022374552,-0.064028427,-0.08015049,-0.1299044960,-0.132518657,0.0009537026,-0.088826778,0.15891712,-0.1069644735,-0.049289219,-0.03885843,-0.11090817,-0.089530808
CMIG4,-0.05935949,-0.042142949,-0.071559455,-0.0640284275,0.433714975,0.13097893,0.0052125469,0.005707779,-0.0653112272,-0.067966260,-0.04847207,0.0471871003,-0.117705647,-0.08414879,-0.01613463,-0.017194341
CPLE6,-0.10136733,-0.056371334,-0.061127738,-0.0801504905,0.130978935,0.48828009,0.0386934716,0.050989796,-0.0581575575,-0.077629332,-0.07884389,0.0242988745,-0.119447735,-0.12448901,0.01369338,0.013429104
ELET3,-0.04223469,-0.101060276,-0.138164604,-0.1299044960,0.005212547,0.03869347,0.5095835891,0.454269785,-0.1031402625,-0.077624280,-0.10071284,-0.0009847393,-0.095262833,-0.09947373,-0.03064707,-0.088225765
ELET6,-0.03766146,-0.097374623,-0.145531732,-0.1325186568,0.005707779,0.05098980,0.4542697853,0.519594976,-0.0659987771,-0.083691096,-0.09543488,-0.0165410303,-0.103700694,-0.11149976,-0.01540778,-0.110504683
EMBR3,0.04665899,-0.073188318,-0.007434768,0.0009537026,-0.065311227,-0.05815756,-0.1031402625,-0.065998777,0.8908647721,0.015966692,0.01902283,-0.0596849595,-0.052353217,-0.07711134,0.08074634,0.043047858
GGBR4,-0.02340204,0.007064551,-0.102137130,-0.0888267780,-0.067966260,-0.07762933,-0.0776242801,-0.083691096,0.0159666917,0.539039423,-0.04424070,0.0063425117,0.009798277,0.02987052,0.01233875,0.009384553


In [32]:
sqrt(mean(modelo_af_pca$residual[lower.tri(modelo_af_pca$residual, diag = FALSE)]^2))

[1] 0.09296638

In [69]:
u_pca = dados - modelo_af_pca$scores %*% t(as.matrix(modelo_af_pca$loadings[1:16, 1]))
u_pa = dados - modelo_af_fp$scores %*% t(as.matrix(modelo_af_fp$loadings[1:16, 1]))
u_ml = dados - modelo_af_mv$scores %*% t(as.matrix(modelo_af_mv$loadings[1:16, 1]))

In [70]:
cov(u_pca)

,ALPA4,BBAS3,BBDC3,BBDC4,CMIG4,CPLE6,ELET3,ELET6,EMBR3,GGBR4,ITSA4,LIGT3,PETR3,PETR4,SBSP3,VIVT3
ALPA4,0.77690144,-0.066627433,-0.058534352,-0.0677730501,-0.059359493,-0.10136733,-0.0422346915,-0.037661461,0.0466589950,-0.023402042,-0.01214240,-0.0464593649,-0.025158099,-0.03484901,0.03955367,0.060282583
BBAS3,-0.06662743,0.322530642,0.018398239,0.0209745280,-0.042142949,-0.05637133,-0.1010602764,-0.097374623,-0.0731883181,0.007064551,0.03304410,-0.0833933717,0.011974459,0.03220912,-0.06802733,-0.017446314
BBDC3,-0.05853435,0.018398239,0.355915338,0.2677344931,-0.071559455,-0.06112774,-0.1381646041,-0.145531732,-0.0074347675,-0.102137130,0.12921416,-0.0948971635,-0.066867936,-0.05712308,-0.08734877,-0.075841845
BBDC4,-0.06777305,0.020974528,0.267734493,0.3022374552,-0.064028427,-0.08015049,-0.1299044960,-0.132518657,0.0009537026,-0.088826778,0.15891712,-0.1069644735,-0.049289219,-0.03885843,-0.11090817,-0.089530808
CMIG4,-0.05935949,-0.042142949,-0.071559455,-0.0640284275,0.433714975,0.13097893,0.0052125469,0.005707779,-0.0653112272,-0.067966260,-0.04847207,0.0471871003,-0.117705647,-0.08414879,-0.01613463,-0.017194341
CPLE6,-0.10136733,-0.056371334,-0.061127738,-0.0801504905,0.130978935,0.48828009,0.0386934716,0.050989796,-0.0581575575,-0.077629332,-0.07884389,0.0242988745,-0.119447735,-0.12448901,0.01369338,0.013429104
ELET3,-0.04223469,-0.101060276,-0.138164604,-0.1299044960,0.005212547,0.03869347,0.5095835891,0.454269785,-0.1031402625,-0.077624280,-0.10071284,-0.0009847393,-0.095262833,-0.09947373,-0.03064707,-0.088225765
ELET6,-0.03766146,-0.097374623,-0.145531732,-0.1325186568,0.005707779,0.05098980,0.4542697853,0.519594976,-0.0659987771,-0.083691096,-0.09543488,-0.0165410303,-0.103700694,-0.11149976,-0.01540778,-0.110504683
EMBR3,0.04665899,-0.073188318,-0.007434768,0.0009537026,-0.065311227,-0.05815756,-0.1031402625,-0.065998777,0.8908647721,0.015966692,0.01902283,-0.0596849595,-0.052353217,-0.07711134,0.08074634,0.043047858
GGBR4,-0.02340204,0.007064551,-0.102137130,-0.0888267780,-0.067966260,-0.07762933,-0.0776242801,-0.083691096,0.0159666917,0.539039423,-0.04424070,0.0063425117,0.009798277,0.02987052,0.01233875,0.009384553


In [71]:
cov(u_pa)

,ALPA4,BBAS3,BBDC3,BBDC4,CMIG4,CPLE6,ELET3,ELET6,EMBR3,GGBR4,ITSA4,LIGT3,PETR3,PETR4,SBSP3,VIVT3
ALPA4,7.994526e-01,-0.052249407,-0.043244072,-0.055066134,-0.039442825,-0.07974171,-0.019849575,-0.015054506,0.064420038,-0.0008192859,-8.452997e-05,-0.022337245,-0.006327663,-0.01862772,0.063715949,0.081783499
BBAS3,-5.224941e-02,0.301064538,-0.000159518,-0.004329469,-0.049127621,-0.05798392,-0.100371769,-0.095850292,-0.059420028,0.0093754646,5.772431e-03,-0.073657613,0.002773484,0.01549077,-0.059079260,-0.002232517
BBDC3,-4.324407e-02,-0.000159518,0.340126106,0.245481822,-0.076243438,-0.06069578,-0.135538826,-0.142113898,0.006866344,-0.0979946091,1.050856e-01,-0.083756930,-0.073696428,-0.07112986,-0.076932820,-0.059872624
BBDC4,-5.506613e-02,-0.004329469,0.245481822,0.272968482,-0.074270384,-0.08475941,-0.132104171,-0.133836102,0.013627070,-0.0892832629,1.275879e-01,-0.099527029,-0.061804892,-0.05922793,-0.104334987,-0.075774046
CMIG4,-3.944283e-02,-0.049127621,-0.076243438,-0.074270384,0.438657795,0.14018274,0.016254521,0.017391845,-0.048054526,-0.0557882427,-6.028566e-02,0.064864777,-0.114701232,-0.08721004,0.001055942,0.002762023
CPLE6,-7.974171e-02,-0.057983918,-0.060695779,-0.084759411,0.140182737,0.50127568,0.053330799,0.066188951,-0.039896626,-0.0620501803,-8.485640e-02,0.044593555,-0.112051633,-0.12253793,0.033617694,0.034804372
ELET3,-1.984958e-02,-0.100371769,-0.135538826,-0.132104171,0.016254521,0.05333080,0.525780468,0.470994780,-0.084426134,-0.0605676710,-1.042458e-01,0.020457084,-0.085974767,-0.09537137,-0.009526022,-0.066215657
ELET6,-1.505451e-02,-0.095850292,-0.142113898,-0.133836102,0.017391845,0.06618895,0.470994780,0.536834528,-0.047165339,-0.0661383469,-9.805677e-02,0.005266018,-0.093746681,-0.10662443,0.006097327,-0.088317193
EMBR3,6.442004e-02,-0.059420028,0.006866344,0.013627070,-0.048054526,-0.03989663,-0.084426134,-0.047165339,0.904658106,0.0347170759,3.130901e-02,-0.040236334,-0.035830433,-0.06217639,0.100291493,0.059861590
GGBR4,-8.192859e-04,0.009375465,-0.097994609,-0.089283263,-0.055788243,-0.06205018,-0.060567671,-0.066138347,0.034717076,0.5568843964,-4.596025e-02,0.028271438,0.020287300,0.03543693,0.033986984,0.031509559


In [72]:
cov(u_ml)

,ALPA4,BBAS3,BBDC3,BBDC4,CMIG4,CPLE6,ELET3,ELET6,EMBR3,GGBR4,ITSA4,LIGT3,PETR3,PETR4,SBSP3,VIVT3
ALPA4,0.818772866,-0.03362975,-0.058932698,-0.066960807,-0.0038908398,-0.043473875,0.02636420,0.030921910,0.071188010,0.0299207594,0.000366736,0.014370170,0.021166750,0.01238798,0.09778028,0.10415681
BBAS3,-0.033629747,0.30157127,-0.065982732,-0.064443752,-0.0116382026,-0.017234178,-0.03933937,-0.034935768,-0.059446788,0.0410211287,-0.030753414,-0.026622685,0.024194213,0.04152249,-0.01789760,0.02444643
BBDC3,-0.058932698,-0.06598273,0.205436188,0.113679257,-0.0957308881,-0.073290535,-0.12510544,-0.131165380,-0.017672797,-0.1170821481,-0.002750955,-0.079665115,-0.110784208,-0.10753729,-0.08030761,-0.06316208
BBDC4,-0.066960807,-0.06444375,0.113679257,0.144556698,-0.0871562090,-0.090914527,-0.11453720,-0.115813659,-0.008812128,-0.1025721180,0.024190560,-0.089599199,-0.092895972,-0.08909046,-0.10200365,-0.07527231
CMIG4,-0.003890840,-0.01163820,-0.095730888,-0.087156209,0.5034815066,0.205848102,0.09824246,0.098922491,-0.034388643,0.0004040175,-0.052616271,0.130254442,-0.063168215,-0.02942259,0.06219368,0.04298530
CPLE6,-0.043473875,-0.01723418,-0.073290535,-0.090914527,0.2058481022,0.567430330,0.13459968,0.146975401,-0.025014569,-0.0050167426,-0.071722062,0.109624763,-0.058751558,-0.06303241,0.09478637,0.07510237
ELET3,0.026364202,-0.03933937,-0.125105445,-0.114537202,0.0982424608,0.134599682,0.62072129,0.565262959,-0.062042219,0.0110511989,-0.068205218,0.097232993,-0.015406372,-0.01753160,0.06405483,-0.01753904
ELET6,0.030921910,-0.03493577,-0.131165380,-0.115813659,0.0989224910,0.146975401,0.56526296,0.630432275,-0.024822930,0.0050900165,-0.061786721,0.081516824,-0.023483793,-0.02914116,0.07920656,-0.03994798
EMBR3,0.071188010,-0.05944679,-0.017672797,-0.008812128,-0.0343886427,-0.025014569,-0.06204222,-0.024822930,0.904569151,0.0462442477,0.017585279,-0.022998794,-0.028105728,-0.05276093,0.11536146,0.06962089
GGBR4,0.029920759,0.04102113,-0.117082148,-0.102572118,0.0004040175,-0.005016743,0.01105120,0.005090017,0.046244248,0.6055602290,-0.040960107,0.085325180,0.064621302,0.08523256,0.08721839,0.06651447


Mais detalhes dos outputs podem ser encontrados [aqui](https://m-clark.github.io/posts/2020-04-10-psych-explained/)

## Método do Factor Princial (PA)



Por simplicidade, trabalharemos com a matriz de correlação mas o mesmo procedimento pode ser aplicado caso utilizemos a matriz de covariância.

1. Suponha que conhecemos $\boldsymbol{\Psi}$, então $\Lambda \Lambda' = \textbf{R} - \boldsymbol{\Psi}$
2. Podemos aplicar a decomposição espectral em $\textbf{R} - \boldsymbol{\Psi} = \textbf{R}^{\ast}$
3. Então $\textbf{R}^{\ast} \approx \Lambda^{\ast}_k \Lambda^{\ast \prime}_k$ (consideramos apenas as $k$ primeiras componentes)
4. Então, $\boldsymbol{\Psi} \approx Diag\{\textbf{R}^{\ast}  -  \Lambda^{\ast}_k \Lambda^{\ast \prime}_k\}$ 
5. Na prática, não conhecemos $\boldsymbol{\Psi}$ no passo 1, mas podemos, para um valor inicial, fazer 2--4 até alcançar convergência.

> Existem vários métodos para o valor inicial de $\boldsymbol{\Psi}$. Um deles é fazer $\psi_i = 1/r^{ii}$ ou $psi_i = 1/s^{ii}$ (dependendo se estamos utilizando a matriz de correlação ou matriz de covariância), em que $r^{ii}$ é o elemento i-éssimo da diagiona de $\textbf{R}^{-1}$ e $s^{ii}$ é o elemento i-éssimo da diagiona de $\textbf{S}^{-1}$
